<a href="https://colab.research.google.com/github/ten-jampa/LLM_grind/blob/main/course/en/chapter3/section4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A full training

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [2]:
!pip install -U datasets

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [10]:
# prompt: how do I get the dataset informatioin (meta information of the dataset) of raw_datasets

train_ds = raw_datasets['train']
train_df = pd.DataFrame(train_ds)
train_df.head(10)

,sentence1,sentence2,label,idx
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr...",1,0
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...,0,1
2,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ...",1,2
3,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set...",0,3
4,"The stock rose $ 2.11 , or about 11 percent , ...",PG & E Corp. shares jumped $ 1.63 or 8 percent...,1,4
5,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart 's compa...,1,5
6,"The Nasdaq had a weekly gain of 17.27 , or 1.2...",The tech-laced Nasdaq Composite .IXIC rallied ...,0,6
7,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,1,7
8,"That compared with $ 35.18 million , or 24 cen...",Earnings were affected by a non-recurring $ 8 ...,0,8
9,"Shares of Genentech , a much larger company wi...",Shares of Xoma fell 16 percent in early trade ...,0,10


In [59]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [8]:
import pandas as pd
tok_df_train = pd.DataFrame(tokenized_datasets['train'])
tok_df_train.head(10)

,sentence1,sentence2,label,idx,input_ids,token_type_ids,attention_mask
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr...",1,0,"[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...,0,1,"[101, 9805, 3540, 11514, 2050, 3079, 11282, 22...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ...",1,2,"[101, 2027, 2018, 2405, 2019, 15147, 2006, 199...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set...",0,3,"[101, 2105, 6021, 19481, 13938, 2102, 1010, 21...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"The stock rose $ 2.11 , or about 11 percent , ...",PG & E Corp. shares jumped $ 1.63 or 8 percent...,1,4,"[101, 1996, 4518, 3123, 1002, 1016, 1012, 2340...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart 's compa...,1,5,"[101, 6599, 1999, 1996, 2034, 4284, 1997, 1996...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,"The Nasdaq had a weekly gain of 17.27 , or 1.2...",The tech-laced Nasdaq Composite .IXIC rallied ...,0,6,"[101, 1996, 17235, 2850, 4160, 2018, 1037, 488...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,1,7,"[101, 1996, 4966, 1011, 10507, 2050, 2059, 120...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,"That compared with $ 35.18 million , or 24 cen...",Earnings were affected by a non-recurring $ 8 ...,0,8,"[101, 2008, 4102, 2007, 1002, 3486, 1012, 2324...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,"Shares of Genentech , a much larger company wi...",Shares of Xoma fell 16 percent in early trade ...,0,10,"[101, 6661, 1997, 4962, 10111, 2818, 1010, 103...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [11]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"]) #because we don't know the text sentences anymore
tokenized_datasets = tokenized_datasets.rename_column("label", "labels") #rename label to labels
tokenized_datasets.set_format("torch") #set all numerical and array values to torch
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [12]:
import pandas as pd
tok_df_train = pd.DataFrame(tokenized_datasets['train'])
tok_df_train.head(10)

,labels,input_ids,token_type_ids,attention_mask
0,tensor(1),"[tensor(101), tensor(2572), tensor(3217), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
1,tensor(0),"[tensor(101), tensor(9805), tensor(3540), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
2,tensor(1),"[tensor(101), tensor(2027), tensor(2018), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
3,tensor(0),"[tensor(101), tensor(2105), tensor(6021), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
4,tensor(1),"[tensor(101), tensor(1996), tensor(4518), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
5,tensor(1),"[tensor(101), tensor(6599), tensor(1999), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
6,tensor(0),"[tensor(101), tensor(1996), tensor(17235), ten...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
7,tensor(1),"[tensor(101), tensor(1996), tensor(4966), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
8,tensor(0),"[tensor(101), tensor(2008), tensor(4102), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
9,tensor(0),"[tensor(101), tensor(6661), tensor(1997), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."


In [13]:
["attention_mask", "input_ids", "labels", "token_type_ids"] # features

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

## Define our dataloaders

In [14]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator # collator prepares batches of data, padds sequences to the same length
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [21]:
import os
os.cpu_count()

2

In [25]:
!nvidia-smi

Wed Jul  2 23:31:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [26]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()} # each batch contains the matrices of data

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 78]),
 'token_type_ids': torch.Size([8, 78]),
 'attention_mask': torch.Size([8, 78])}

In [37]:
# To probe deeper, let's convert the batch to a dataframe
sample_batch_df = pd.DataFrame()
for k, v in batch.items():
  sample_batch_df[k] = v.tolist() # Convert tensors to lists

sample_batch_df.head()

,labels,input_ids,token_type_ids,attention_mask
0,0,"[101, 1996, 10754, 2080, 10506, 1998, 6788, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,0,"[101, 1996, 23268, 3557, 3919, 2779, 3092, 209...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1,"[101, 2002, 2036, 2056, 2009, 2003, 2205, 2220...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,0,"[101, 1000, 2023, 2003, 2200, 3809, 1010, 1000...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1,"[101, 7243, 15794, 1010, 1997, 8242, 2239, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [38]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2) #

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# finetuning to classify whether two sentences are the same or not
We take the pretrained Sequence classification ```bert-base-uncased``` off the hugging face hub and instantiate it to train for end label of 2. This removes the head and instantiates a new head for the new classification, whose final weights we have to train.

In [39]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.8160, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [41]:
print(outputs)

SequenceClassifierOutput(loss=tensor(0.8160, grad_fn=<NllLossBackward0>), logits=tensor([[1.1914, 0.1936],
        [1.2096, 0.2140],
        [1.2068, 0.1954],
        [1.1937, 0.2025],
        [1.2047, 0.2025],
        [1.2066, 0.1950],
        [1.2102, 0.2024],
        [1.2105, 0.2030]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


## importing Optimizer

In [47]:
from torch.optim import AdamW # the optimizer we have used Adam but with a twist for weight decay regularization

optimizer = AdamW(model.parameters(), lr=5e-5)

In [50]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
print(f'num_training_steps: {num_training_steps}, \n num_epochs: {num_epochs}, \n len(train_dataloader): {len(train_dataloader)}')
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


num_training_steps: 1377, 
 num_epochs: 3, 
 len(train_dataloader): 459


In [51]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [52]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()} # move the input features onto the gpu
        outputs = model(**batch) # do the forward pass of the model
        loss = outputs.loss # evaluate the loss as compared to the true data
        loss.backward() # do the backprop to evaluate the gradients

        optimizer.step() # use the optimizer to update the step
        lr_scheduler.step() # step according to the lr scehdular
        optimizer.zero_grad() # clean the gradients
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [53]:
?model

In [123]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
first_flag = True
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    if first_flag:
      print(logits[0])
      print(predictions)
      first_flag = False
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

tensor([-2.3096,  3.6031], device='cuda:0')
tensor([1, 0, 0, 1, 0, 1, 1, 1], device='cuda:0')


{'accuracy': 0.8357843137254902, 'f1': 0.8834782608695653}

In [121]:
?metric

In [61]:
from transformers import AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        # Gradient Clippings
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1377 [00:00<?, ?it/s]

In [73]:
import datasets

In [75]:
# soft_eval
from datasets import Dataset

test_sentence1 = 'He said that the weather was not good today!'
test_sentence2 = 'The weather was very bad today, he said.'
test_ds = Dataset.from_dict({
    'sentence1': [test_sentence1],
    'sentence2': [test_sentence2]
})
test_ds

Dataset({
    features: ['sentence1', 'sentence2'],
    num_rows: 1
})

In [103]:
def get_tokenised_sentences(sentence1, sentence2):
  return tokenizer(sentence1, sentence2, truncation= True)


In [83]:
tokenized_test_ds = test_ds.map(tokenize_function, batched=True)
tokenized_test_ds

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence1', 'sentence2', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1
})

In [84]:
tokenized_test_df = pd.DataFrame(tokenized_test_ds)
tokenized_test_df.head()

## remove the sentence column

,sentence1,sentence2,input_ids,token_type_ids,attention_mask
0,He said that the weather was not good today!,"The weather was very bad today, he said.","[101, 2002, 2056, 2008, 1996, 4633, 2001, 2025...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [79]:
# Tokenize the two sentences directly
tokenized_sentences = tokenizer(test_sentence1, test_sentence2, truncation=True)
tokenized_sentences

{'input_ids': [101, 2002, 2056, 2008, 1996, 4633, 2001, 2025, 2204, 2651, 999, 102, 1996, 4633, 2001, 2200, 2919, 2651, 1010, 2002, 2056, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [88]:
# The tokenizer returns a dictionary. To use data_collator, we need to put it in a list
# as data_collator expects a list of dictionaries (even if there's only one item).
tokenized_sentences_list = [tokenized_sentences]

# Prepare the tokenized sentences as a batch using the data collator
inference_batch = data_collator(tokenized_sentences_list) # converts to tensor objects from here

# Move the batch to the device
inference_batch = {k: v.to(device) for k, v in inference_batch.items()}

inference_batch

{'input_ids': tensor([[ 101, 2002, 2056, 2008, 1996, 4633, 2001, 2025, 2204, 2651,  999,  102,
          1996, 4633, 2001, 2200, 2919, 2651, 1010, 2002, 2056, 1012,  102]],
        device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
        device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
        device='cuda:0')}

In [104]:
def make_inference_batch(tokenized_sentences):
  tokenized_sentences_list = [tokenized_sentences]

  # Prepare the tokenized sentences as a batch using the data collator
  inference_batch = data_collator(tokenized_sentences_list) # converts to tensor objects from here

  # Move the batch to the device
  inference_batch = {k: v.to(device) for k, v in inference_batch.items()}

  return inference_batch


In [101]:
with torch.no_grad():
  test_output = model(**inference_batch)
print(test_output.logits)

# we have to do a softmax now on the logits
from torch.nn.functional import softmax

probabilities = softmax(test_output.logits, dim=-1) # Added dim=-1
print(probabilities)
ids = torch.argmax(probabilities, axis = 1)
print(ids)

# Map the predicted index to the label
# For MRPC, class 0 is 'not_equivalent' and class 1 is 'equivalent'
class_labels = ["not_equivalent", "equivalent"]
predicted_label = class_labels[ids.item()]
print(f"Predicted label: {predicted_label}")

tensor([[-2.8951,  3.3674]], device='cuda:0')
tensor([[0.0019, 0.9981]], device='cuda:0')
tensor([1], device='cuda:0')
Predicted label: equivalent


In [129]:
def inference(model, inference_batch, verbose = True):
  with torch.no_grad():
    test_output =model(**inference_batch)
  if verbose:
    print(test_output)

  # probs = torch.nn.functional.softmax(test_output.logits, dim = -1)
  output_ids = torch.argmax(test_output.logits, axis = -1)
  print(output_ids)
  # Map the predicted index to the labe
  class_labels = ['Not equivalent', 'Equivalent']
  predicted_label = class_labels[output_ids.item()]
  return predicted_label

In [127]:
# Alright, since we have a working infernece loop, let's code it up

def inference_loop(model, sentences):
  sentence1, sentence2 = sentences

  tokenized_sentences = get_tokenised_sentences(sentence1, sentence2)

  inference_batch = make_inference_batch(tokenized_sentences)

  predicted_labels = inference(model, inference_batch)

  return predicted_labels

In [130]:
# test
new_sentence_1 = 'I think physics is the best subject'
new_sentence_2 = 'I think maths is the worst subject'

print(inference_loop(model, [new_sentence_1, new_sentence_2])) # it's fairly bad at this pt

SequenceClassifierOutput(loss=None, logits=tensor([[-1.9505,  3.4004]], device='cuda:0'), hidden_states=None, attentions=None)
tensor([1], device='cuda:0')
Equivalent


# Using the Accelerate library to enable distributed training on multiple GPUs or TPUs

In [116]:
from accelerate import Accelerator
from transformers import AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1377 [00:00<?, ?it/s]

In [131]:
def training_function():
  from accelerate import Accelerator
  from transformers import AutoModelForSequenceClassification, get_scheduler
  from torch.optim import AdamW

  accelerator = Accelerator()

  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
  optimizer = AdamW(model.parameters(), lr=3e-5)

  train_dl, eval_dl, model, optimizer = accelerator.prepare(
      train_dataloader, eval_dataloader, model, optimizer
  )

  num_epochs = 3
  num_training_steps = num_epochs * len(train_dl)
  lr_scheduler = get_scheduler(
      "linear",
      optimizer=optimizer,
      num_warmup_steps=0,
      num_training_steps=num_training_steps,
  )

  progress_bar = tqdm(range(num_training_steps))

  model.train()
  for epoch in range(num_epochs):
      for batch in train_dl:
          outputs = model(**batch)
          loss = outputs.loss
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
  return model


In [132]:
from accelerate import notebook_launcher

notebook_launcher(training_function)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Launching training on one GPU.


  0%|          | 0/1377 [00:00<?, ?it/s]

In [23]:
import torch

if torch.cuda.is_available():
    print(f"Number of GPU devices available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU devices available.")

Number of GPU devices available: 1
Device 0: Tesla T4


In [133]:
!nvidia-smi

Thu Jul  3 00:57:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P0             69W /   70W |    4378MiB /  15360MiB |     97%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----